In [1]:
#! pip install langchain openai chromadb langchainhub

# Set env var OPENAI_API_KEY or load from a .env file
import dotenv
import os
import openai
from dotenv import load_dotenv, find_dotenv

env_path = os.path.join(os.getcwd(), '.env')
# Load env vars from .env file
load_dotenv(find_dotenv())

# Set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:

from langchain.document_loaders import WebBaseLoader

#loader = WebBaseLoader("https://saychess.substack.com/p/10-simple-pieces-of-chess-advice")
loader= WebBaseLoader('https://www.rillavoice.com/#how-it-works') 
# Split documents


In [4]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
splits = text_splitter.split_documents(loader.load())


In [5]:

# Embed and store splits

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
vectorstore = Chroma.from_documents(documents=splits,embedding=OpenAIEmbeddings(openai_api_key=openai.api_key))
retriever = vectorstore.as_retriever()

# Prompt 
# https://smith.langchain.com/hub/rlm/rag-prompt

from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")


In [6]:

# LLM

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1 , openai_api_key=openai.api_key,  )

# RAG chain 

from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | rag_prompt 
    | llm 
)


In [7]:

rag_chain.invoke("how does Rillavoice works?")

AIMessage(content="Rillavoice works by capturing conversations from sales representatives' phones or tablets using the Rilla App. It then automatically transcribes and analyzes these conversations using Artificial Intelligence. This allows sales teams to train and improve by analyzing the thousands of conversations they have with customers in the field.")